In [11]:
import pandas as pd 
import numpy as np 
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import re
import time

In [18]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

keyword = "AWS"
page = 1
main_url = "https://www.rallit.com"
rallit_res = pd.DataFrame()

# 웹 드라이버 초기화
driver = webdriver.Chrome()  # 혹은 다른 브라우저에 맞게 변경

while True:
    url = f'https://www.rallit.com/?jobSkillKeywords={keyword}&pageNumber={page}'
    driver.get(url)
    time.sleep(3)  # 페이지가 완전히 로드될 때까지 잠시 대기

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    if "검색결과가 없어요" in soup.text:
        break

    # 각 공고를 클릭하여 정보 수집
    job_elements = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[3]/ul/li/article/a/figure/img')
    for i in range(len(job_elements)):
        job_elements = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[3]/ul/li/article/a/figure/img')
        job_element = job_elements[i]
        job_element.click()

        # 공고 페이지가 완전히 로드될 때까지 대기
        WebDriverWait(driver, 10).until(EC.url_changes(driver.current_url))

        sub_soup = BeautifulSoup(driver.page_source, 'html.parser')

        corp_name = sub_soup.select_one('h2').text.strip() if sub_soup.select_one('h2') else "" # 기업명
        address = sub_soup.select_one('p.css-1hdnp7d').text.strip() if sub_soup.select_one('p.css-1hdnp7d') else ""  # 주소
        career = sub_soup.select_one('dd.css-1pvdrt3').text.strip() if sub_soup.select_one('dd.css-1pvdrt3') else ""  # 직급
        job_description = sub_soup.select('h1.css-17ueevk')[0].text.strip()  # 직무
        job_skill = [keyword.text.strip()[1:] if keyword.text.strip().startswith('#') else keyword.text.strip() for keyword in sub_soup.select('div.css-175wty3')]
        job_preferential = sub_soup.select('p.css-19hzmb1')[4].text.strip() if len(sub_soup.select('p.css-19hzmb1')) > 4 else ""  # 우대사항

        employment_url = driver.current_url #해당 페이지 URL
        job_requirements = sub_soup.select('p.css-19hzmb1')[3].text.strip() if len(sub_soup.select('p.css-19hzmb1')) > 3 else ""  # 자격요건

        temp_dict = {
            '기업명': corp_name,
            '주소': address,
            '직무': job_description,
            '직급': career,
            '이용하는 기술스택': job_skill,
            '우대사항': job_preferential,
            '해당 페이지 URL': employment_url,
            '자격요건': job_requirements
        }

        temp_df = pd.DataFrame([temp_dict])
        rallit_res = pd.concat([rallit_res, temp_df], ignore_index=True)

        # 뒤로 가기
        driver.back()
        time.sleep(3)  # 페이지가 완전히 로드될 때까지 잠시 대기

    page += 1

# 크롤링이 끝나면 드라이버 종료
driver.quit()

# 결과 확인
print(rallit_res)


KeyboardInterrupt: 

In [15]:
# 결과를 CSV 파일로 저장
rallit_res.to_csv('rallit_0.csv', index=False, encoding='utf-8-sig')

In [14]:
import os
print(os.getcwd())


c:\Users\user\Downloads\crawling_project
